### Meant to be for Task 5
#### Untested and high erroneous

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import zipfile as zp
import os

In [4]:
num_of_labels = 10

# Assign Zip Files to variable
zf = zp.ZipFile('fashion-mnist_train.zip')
zf1 = zp.ZipFile('fashion-mnist_test.zip')

# Load CSV files from Zip Files
train_data = np.loadtxt(zf.open('fashion-mnist_train.csv'), delimiter=',')
test_data = np.loadtxt(zf1.open('fashion-mnist_test.csv'), delimiter=',')

# Display images from 1 to 10
# for i in range(10):
#     img = train_imgs[i].reshape((28,28))
#     plt.imshow(img, cmap="Greys")
#     plt.show()

# Map image data values into intervals [0.01, 0.99]
fac = 0.99 / 255
add_fac = 0.01
train_imgs = np.asfarray(train_data[:, 1:]) * fac + add_fac
test_imgs = np.asfarray(test_data[:, 1:]) *fac + add_fac
train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])

lr = np.arange(num_of_labels)
# transform labels into one hot representation
train_labels_one_hot = (lr==train_labels).astype(np.float64)
test_labels_one_hot = (lr==test_labels).astype(np.float64)
# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99

# Create Pickle file from previous data
with open(os.path.join(".","pkl_fashionmnist.pkl"), "bw") as fh:
    data = (train_imgs, 
            test_imgs, 
            train_labels,
            test_labels,
            train_labels_one_hot,
            test_labels_one_hot)
    pickle.dump(data, fh)

In [5]:
# Activation Functions
def sigmoid(x):
    return 1 / (1 + np.e ** -x)

# Softmax
def softmax(x):
    expo = np.exp(x)
    expo_sum = np.sum(np.exp(x))
    return expo / expo_sum

class NeuralNetwork:
    def __init__(self,
                train_data,
                train_labels,
                test_data,
                test_labels,
                num_of_input_nodes,
                num_of_hidden_nodes,
                active_input_percentage = 0.70,
                active_hidden_percentage = 0.70,):
        
        self.train_data = train_data
        self.train_labels = train_labels
        self.test_data = test_data
        self.test_labels = test_labels
        self.num_of_input_nodes = num_of_input_nodes
        self.num_of_hidden_nodes = num_of_hidden_nodes
        self.active_input_percentage = active_input_percentage
        self.active_hidden_percentage = active_hidden_percentage
        
        self.use_momentum = True
        
        self.epoch_nums = 100
        
        # default parameters
        # self.num_of_hidden_nodes = 256
#         self.num_of_epochs = 200
#         self.batch_size = 100
#         self.learning_rate = 1
#         self.momentum = 0.9
        
        self.w0 = np.random.randn(self.train_data.shape[1], self.num_of_hidden_nodes) / np.sqrt(self.train_data.shape[1])
        self.w1 = np.random.randn(self.num_of_hidden_nodes, len(self.train_labels)) / np.sqrt(self.num_of_hidden_nodes)
    
        self.v0 = np.zeros(self.w0.shape)
        self.v1 = np.zeros(self.w1.shape)
        
    def dropout_weight_matrices(self):
        # restore wih array, if it had been used for dropout
        self.wih_orig = self.wih.copy()
        self.no_of_in_nodes_orig = self.no_of_in_nodes
        self.no_of_hidden_nodes_orig = self.no_of_hidden_nodes
        self.who_orig = self.who.copy()
        
        active_input_nodes = int(self.num_of_input_nodes * self.active_input_percentage)
        active_input_indices = sorted(random.sample(range(0, self.num_of_input_nodes), 
                                      active_input_nodes))
        active_hidden_nodes = int(self.num_of_hidden_nodes * self.active_hidden_percentage)
        active_hidden_indices = sorted(random.sample(range(0, self.num_of_hidden_nodes), 
                                       active_hidden_nodes))
        
        self.wih = self.wih[:, active_input_indices][active_hidden_indices]       
        self.who = self.who[:, active_hidden_indices]
        
        self.no_of_hidden_nodes = active_hidden_nodes
        self.no_of_in_nodes = active_input_nodes
        return active_input_indices, active_hidden_indices
        
    def train(self, epoch_nums, batch_size, learning_rate, momentum, use_dropout):
        Loss_list = []
        lr = learning_rate
        self.epoch_nums = epoch_nums
        self.use_dropout = use_dropout
        
        for epoch in range(self.epoch_nums):
            if epoch > 0:
                lr *= 0.99
            loss_epoch = []
            
            new_data = self.train_data
            new_labels = self.train_labels
            
            if self.use_dropout == True:
                new_data, new_labels = dropout_weight_matrices()
            
            for imgs, labels in zip(new_data, new_labels):
                
                # Forward pass
                input_vector = sigmoid(np.matmul(imgs, self.w0))
                log = np.matmul(input_vector, self.w1)
                
                soft = softmax(input_vector)
                soft = np.clip(soft, 1e-16, 1)
                
                loss = -np.log(soft[np.arange(0, soft.shape[0]), labels])
                
                # Backward pass
                delta_log = soft
                delta_log[np.arange(0, delta_log.shape[0]), labels] -= 1
                delta_log /= delta_log.shape[0]
                
                delta_w1 = np.matmul(input_vector.T, delta_log)
                delta_l1 = np.matmul(delta_log, self.w1.T)
                delta_l1 *= sigmoid(input_vector) * 1 - sigmoid(input_vector)
                delta_w0 = np.matmul(imgs.T, delta_l1)
                
                if not self.use_momentum:
                    self.w0 -= lr * delta_w0
                    self.w1 -= lr * delta_w1
                else:
                    self.v0 = momentum * self.v0 + (1 - momentum) * delta_w0
                    self.v1 = momentum * self.v1 + (1 - momentum) * delta_w1
                    self.w0 -= lr * self.v0
                    self.w1 -= lr * self.v1
                    
                if len(loss_epoch) == 0:
                    loss_epoch = loss
                else:
                    loss_epoch = np.concatenate([loss_epoch, loss], axis = 0)
            
            mean = np.mean(loss_epoch)
            Loss_list.append(mean)
            if(epoch + 1) % 5 == 0:
                print('Epoch: ' + (epoch + 1) + ' Loss: ' + np.mean(loss_epoch))
        
        return Loss_list
            
    def test(self):
        total_correct = 0.0
        imgs_batch, labels_batch = get_batch(self.input_test, self.target_test, self.batch_size)
        
        for imgs, labels in zip(self.test_data, self.test_labels):
            input_vector = self.sigmoid(np.matmul(imgs))
                                        
            if self.use_dropout == True:
                log = log * self.active_input_percentage
                                        
            log = np.matmul(layer1, self.w1)
            soft = softmax(log)
            pred = np.argmax(soft, axis = 1)
            print('pred: ', pred)
            print('labels: ', labels)
            correct = len(np.where(pred == labels)[0])
            print("correct: ", correct)
            total_correct += correct
            print('Total correct: ', total_correct/self.input_test.shape[0])
                                        

In [6]:
# Load data from Pickle file
with open(os.path.join(".","pkl_fashionmnist.pkl"), "br") as fh:
    data = pickle.load(fh)
train_imgs = data[0]
test_imgs = data[1]
train_labels = data[2]
test_labels = data[3]
train_labels_one_hot = data[4]
test_labels_one_hot = data[5]

img_size = 28 # dimensions
num_of_labels = 10 # 0, 1, 2, ... 9
image_pixels = img_size * img_size

# Run 100 times
epochs = 100

num_of_input_nodes = image_pixels
num_of_hidden_nodes = 256

batch_size = 100
learning_rate = 1
momentum = 0.9
use_dropout = False

NN = NeuralNetwork(train_imgs, 
                   train_labels, 
                   test_imgs, 
                   test_labels, 
                   num_of_hidden_nodes, 
                   num_of_input_nodes)

Loss = NN.train(epochs, batch_size, learning_rate, momentum, use_dropout)
plt.plot(np.arange(NN.num_epochs), Loss)

NN.test()


# learning_rate = 0.5
# Loss = NN.train(epochs, batch_size, learning_rate, momentum)



IndexError: arrays used as indices must be of integer (or boolean) type